# CBA - RG 

In [1]:
'''
Definition 
Find rulesets that have support above minsup
Rule-Item {condset, y}
condsUpCount - Support Count of condset refers to the number of cases that contains condset 
rulesUpCount - The support count of rule item refers to number of cases that contains condset with label y
Support- rulesUpCount/datasize
Confidence - rulesUpCount/condsUpCount
Frequent dataset - If rule item satisfy > minSup (which is rulesUpCount/datasize)
'''

'\nDefinition \nFind rulesets that have support above minsup\nRule-Item {condset, y}\ncondsUpCount - Support Count of condset refers to the number of cases that contains condset \nrulesUpCount - The support count of rule item refers to number of cases that contains condset with label y\nSupport- rulesUpCount/datasize\nConfidence - rulesUpCount/condsUpCount\nFrequent dataset - If rule item satisfy > minSup (which is rulesUpCount/datasize)\n'

In [2]:
'''
For all rule item that have same condset, the rule item with highest confidence (rulesUpCount/condsUpCount) is chosen
If there is a tie, randomly select one rule-item (Maybe consider something else instead of random - Use the one with higher Support)
If confidence of chosen rule is greater than minConf, we say the rule is accurate 
'''

'\nFor all rule item that have same condset, the rule item with highest confidence (rulesUpCount/condsUpCount) is chosen\nIf there is a tie, randomly select one rule-item (Maybe consider something else instead of random - Use the one with higher Support)\nIf confidence of chosen rule is greater than minConf, we say the rule is accurate \n'

# Importing the dataset & Preprocessing

In [3]:
import numpy as np
import pandas as pd
import itertools    


In [4]:
from google.colab import drive
drive.mount('/content/gdrive') 

Mounted at /content/gdrive


# Preprocesing Helper Functions

In [5]:
"""
Description: Recursive minimal entropy partitioning, to discretize continuous-valued attributes. We use the supervised
    algorithm presented in Fayyad & Irani (1993) and introduced in Dougherty, Kohavi & Sahami (1995) section 3.3.
    We also refer to a F# code on GitHub (https://gist.github.com/mathias-brandewinder/5650553).
Input: a data table with several rows but only two column, the first column is continuous-valued (numerical) attributes,
    and the second column is the class label of each data case (categorical).
    e.g. data = [[1.0, 'Yes'], [0.5, 'No'], [2.0, 'Yes']]
Output: a list of partition boundaries of the range of continuous-valued attribute in ascending sort order.
    e.g. walls = [0.5, 0.8, 1.0], thus we can separate the range into 4 intervals: <=0.5, 0.5<*<=0.8, 0.8<*<=1.0 & >=1.0
Author: CBA Studio
Reference:
    1. Multi-Interval Discretization of Continuous-Valued Attributes for Classification Learning, Fayyad & Irani, 1993
    2. Supervised and Unsupervised Discretization of Continuous Features, Dougherty, Kohavi & Sahami, 1995
    3. http://www.clear-lines.com/blog/post/Discretizing-a-continuous-variable-using-Entropy.aspx
"""
import math


# A block to be split
# It has 4 member:
#   data: the data table with a column of continuous-valued attribute and a column of class label
#   size: number of data case in this table
#   number_of_classes: obviously, the number of class in this table
#   entropy: entropy of dataset
class Block:
    def __init__(self, data):
        self.data = data
        self.size = len(data)
        classes = set([x[1] for x in data])     # get distinct class labels in this table
        self.number_of_classes = len(set(classes))
        self.entropy = calculate_entropy(data)


# Calculate the entropy of dataset
# parameter data: the data table to be used
def calculate_entropy(data):
    number_of_data = len(data)
    classes = set([x[1] for x in data])
    class_count = dict([(label, 0) for label in classes])
    for data_case in data:
        class_count[data_case[1]] += 1      # count the number of data case of each class
    entropy = 0
    for c in classes:
        p = class_count[c] / number_of_data
        entropy -= p * math.log2(p)         # calculate information entropy by its formula, where the base is 2
    return entropy


# Compute Gain(A, T: S) mentioned in Dougherty, Kohavi & Sahami (1995), i.e. entropy gained by splitting original_block
#   into left_block and right_block
# original_block: the block before partition
# left_block: the block split which its value below boundary
# right_block: the block above boundary
def entropy_gain(original_block, left_block, right_block):
    gain = original_block.entropy - \
            ((left_block.size / original_block.size) * left_block.entropy +
            (right_block.size / original_block.size) * right_block.entropy)
    return gain


# Get minimum entropy gain required for a split of original_block into 2 blocks "left" and "right", see Dougherty,
#   Kohavi & Sahami (1995)
# original_block: the block before partition
# left_block: the block split which its value below boundary
# right_block: the block above boundary
def min_gain(original_block, left_block, right_block):
    delta = math.log2(math.pow(3, original_block.number_of_classes) - 2) - \
            (original_block.number_of_classes * original_block.entropy -
             left_block.number_of_classes * left_block.entropy -
             right_block.number_of_classes * right_block.entropy)
    gain_sup = math.log2(original_block.size - 1) / original_block.size + delta / original_block.size
    return gain_sup


# Identify the best acceptable value to split block
# block: a block of dataset
# Return value: a list of (boundary, entropy gain, left block, right block) or
#   None when it's unnecessary to split
def split(block):
    candidates = [x[0] for x in block.data]     # candidates is a list of values can be picked up as boundary
    candidates = list(set(candidates))          # get different values in table
    candidates.sort()                           # sort ascending
    candidates = candidates[1:]                 # discard smallest, because by definition no value is smaller

    wall = []       # wall is a list storing final boundary
    for value in candidates:
        # split by value into 2 groups, below & above
        left_data = []
        right_data = []
        for data_case in block.data:
            if data_case[0] < value:
                left_data.append(data_case)
            else:
                right_data.append(data_case)

        left_block = Block(left_data)
        right_block = Block(right_data)

        gain = entropy_gain(block, left_block, right_block)
        threshold = min_gain(block, left_block, right_block)

        # minimum threshold is met, the value is an acceptable candidate
        if gain >= threshold:
            wall.append([value, gain, left_block, right_block])

    if wall:    # has candidate
        wall.sort(key=lambda wall: wall[1], reverse=True)   # sort descending by "gain"
        return wall[0]      # return best candidate with max entropy gain
    else:
        return None         # no need to split


# Top-down recursive partition of a data block, append boundary into "walls"
# block: a data block
def partition(block):
    walls = []

    # inner recursive function, accumulate the partitioning values
    # sub_block: just a data block
    def recursive_split(sub_block):
        wall_returned = split(sub_block)        # binary partition, get bin boundary
        if wall_returned:                       # still can be spilt
            walls.append(wall_returned[0])      # record this partitioning value
            recursive_split(wall_returned[2])   # recursively process left block
            recursive_split(wall_returned[3])   # recursively split right block
        else:
            return                              # end of recursion

    recursive_split(block)      # call inner function
    walls.sort()                # sort boundaries descending
    return walls



In [6]:
"""
Description: Pre-process original data. Firstly, we process the missing values (donated as '?'), discarding this column
    when missing ratio above 50%, or filling blanks when below. We "guess" missing values by simply filling the mode of
    existing values in the same column. And then, for the numerical attribute, we discretizate it by recursive minimal
    entropy partitioning (see rmep.py). For the categorical attribute, we just replace the label with a
    positive integer. For more information, see [1].
Input: a data table with several data case, many attributes and class label in the last column, a list of the name of
    each attribute, and a list of the type of each column.
Output: a data list without numerical values and "str" categorical values.
Author: CBA Studio
Reference:
    1. http://cgi.csc.liv.ac.uk/~frans/KDD/Software/LUCS-KDD-DN/lucs-kdd_DN.html
"""
# Identify the mode of a list, both effective for numerical and categorical list. When there exists too many modes
#   having the same frequency, return the first one.
# arr: a list need to find mode
def get_mode(arr):
    mode = []
    arr_appear = dict((a, arr.count(a)) for a in arr)   # count appearance times of each key
    if max(arr_appear.values()) == 1:       # if max time is 1
        return      # no mode here
    else:
        for k, v in arr_appear.items():     # else, mode is the number which has max time
            if v == max(arr_appear.values()):
                mode.append(k)
    return mode[0]  # return first number if has many modes


# Fill missing values in column column_no, when missing values ration below 50%.
# data: original data list
# column_no: identify the column No. of that to be filled
def fill_missing_values(data, column_no):
    size = len(data)
    column_data = [x[column_no] for x in data]      # get that column
    while '?' in column_data:
        column_data.remove('?')
    mode = get_mode(column_data)
    for i in range(size):
        if data[i][column_no] == '?':
            data[i][column_no] = mode              # fill in mode
    return data


# Get the list needed by rmep.py, just glue the data column with class column.
# data_column: the data column
# class_column: the class label column
def get_discretization_data(data_column, class_column):
    size = len(data_column)
    result_list = []
    for i in range(size):
        result_list.append([data_column[i], class_column[i]])
    return result_list


# Replace numerical data with the No. of interval, i.e. consecutive positive integers.
# data: original data table
# column_no: the column No. of that column
# walls: the split point of the whole range
def replace_numerical(data, column_no, walls):
    size = len(data)
    num_spilt_point = len(walls)
    for i in range(size):
        if data[i][column_no] > walls[num_spilt_point - 1]:
            data[i][column_no] = num_spilt_point + 1
            continue
        for j in range(0, num_spilt_point):
            if data[i][column_no] <= walls[j]:
                data[i][column_no] = j + 1
                break
    return data


# Replace categorical values with a positive integer.
# data: original data table
# column_no: identify which column to be processed
def replace_categorical(data, column_no):
    size = len(data)
    classes = set([x[column_no] for x in data])
    classes_no = dict([(label, 0) for label in classes])
    j = 1
    for i in classes:
        classes_no[i] = j
        j += 1
    for i in range(size):
        data[i][column_no] = classes_no[data[i][column_no]]
    return data, classes_no


# Discard all the column with its column_no in discard_list
# data: original data set
# discard_list: a list of column No. of the columns to be discarded
def discard(data, discard_list):
    size = len(data)
    length = len(data[0])
    data_result = []
    for i in range(size):
        data_result.append([])
        for j in range(length):
            if j not in discard_list:
                data_result[i].append(data[i][j])
    return data_result


# Main method here, see Description in detail
# data: original data table
# attribute: a list of the name of attribute
# value_type: a list identifying the type of each column
# Returned value: a data table after process
def pre_process(data, attribute, value_type):
    column_num = len(data[0])
    size = len(data)
    class_column = [x[-1] for x in data]
    discard_list = []
    for i in range(0, column_num - 1):
        data_column = [x[i] for x in data]

        # process missing values
        missing_values_ratio = data_column.count('?') / size
        if missing_values_ratio > 0.5:
            discard_list.append(i)
            continue
        elif missing_values_ratio > 0:
            data = fill_missing_values(data, i)
            data_column = [x[i] for x in data]

        # discretization
        if value_type[i] == 'numerical':
            discretization_data = get_discretization_data(data_column, class_column)
            block = Block(discretization_data)
            walls = partition(block)
            if len(walls) == 0:
                max_value = max(data_column)
                min_value = min(data_column)
                step = (max_value - min_value) / 3
                walls.append(min_value + step)
                walls.append(min_value + 2 * step)
            print(attribute[i] + ":", walls)        # print out split points
            data = replace_numerical(data, i, walls)
        elif value_type[i] == 'categorical':
            data, classes_no = replace_categorical(data, i)
            print(attribute[i] + ":", classes_no)   # print out replacement list

    # discard
    if len(discard_list) > 0:
        data = discard(data, discard_list)
        print("discard:", discard_list)             # print out discard list
    return data


# Importing data

In [7]:
iris = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Data Analytics/iris.data', header = None)
bank = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Data Analytics/banknote_authentication.data', header = None)
diabetes = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Data Analytics/pima-indians-diabetes.data', header = None)


In [8]:
# transactions = iris.values
# transactions = bank.values
transactions = diabetes.values

In [9]:
# Iris Dataset
# test_attribute = ['sepal length','sepal width','petal length','petal width','class']
# test_value_type = ['numerical','numerical','numerical','numerical','label']
# test_data_after = pre_process(transactions, test_attribute, test_value_type)
# test_data_after

# Bank Dataset
# test_attribute = ['variance of Wavelet Transformed image','skewness of Wavelet Transformed image','curtosis of Wavelet Transformed image','entropy of image','class']
# test_value_type = ['numerical','numerical','numerical','numerical','label']
# test_data_after = pre_process(transactions, test_attribute, test_value_type)
# test_data_after

# Diabetes Dataset
test_attribute = ['Number of times pregnant','Plasma glucose concentration a 2 hours in an oral glucose tolerance test','Diastolic blood pressure','Triceps skin fold thickness','2-Hour serum insulin','Body mass index','Diabetes pedigree function','Age','class']
test_value_type = ['numerical','numerical','numerical','numerical','numerical','numerical','numerical','numerical','label']
test_data_after = pre_process(transactions, test_attribute, test_value_type)
test_data_after


Number of times pregnant: [7.0]
Plasma glucose concentration a 2 hours in an oral glucose tolerance test: [100.0, 128.0, 155.0]
Diastolic blood pressure: [40.666666666666664, 81.33333333333333]
Triceps skin fold thickness: [33.0, 66.0]
2-Hour serum insulin: [15.0, 122.0]
Body mass index: [27.9]
Diabetes pedigree function: [0.528]
Age: [29.0]


array([[1., 3., 2., ..., 2., 2., 1.],
       [1., 1., 2., ..., 1., 2., 0.],
       [2., 4., 2., ..., 2., 2., 1.],
       ...,
       [1., 2., 2., ..., 1., 2., 0.],
       [1., 2., 2., ..., 1., 2., 1.],
       [1., 1., 2., ..., 1., 1., 0.]])

# CBG - RG Algorithm

In [10]:
class CBA_RG():
  # Generate the large frequent item step
  def __init__(self, min_sup, transactions, confidence):
    self.min_sup = min_sup
    self.min_conf = confidence
    # Pass in an array of transactions - Generate freq 1-itemset
    freq_data_set = {}
    self.transaction_length = len(transactions)
    self.total_rules_seen = 0
    # First Pass
    item_sets, rules_up_count = self.first_pass(transactions)
    car_1 , scores = self.gen_rules(item_sets,rules_up_count)
    myResults = self.get_freq_items_list(item_sets, True)
    self.first = myResults
    self.all_car,self.get_all_scores = self.next_pass(myResults, transactions)
    self.car_first = car_1
    self.scores = scores
    # <(condset, condsupCount), (y, rulesupCount)>
    # Optional Rule Pruning could be done here
  
  # Rule Itemset -> {column_cat, value, label : count}
  def first_pass(self, data):
    item_sets = {}
    rules_up_count = {}
    for transaction in data:
      label = transaction[-1]
      for col in range(len(transaction)-1):
        # <(condset, condsupCount), (y, rulesupCount)>
        itemset = (col, transaction[col])
        temp = item_sets.get(itemset, 0) + 1 # update the cond count
        tempRuleUpCount = rules_up_count.get((itemset,label), 0) + 1 # update the rules up count
        item_sets[itemset] = temp
        rules_up_count[(itemset,label)] = tempRuleUpCount
    for x in rules_up_count.copy():
      if (rules_up_count[x]/self.transaction_length) < self.min_sup: # There could be 2 rules that has same cond 
        del rules_up_count[x]
  
    # We look for the cond support for item_sets now -> If rules_up_count doesn't have it, we delete freq
    for x in item_sets.copy():
      delete = True
      for y in rules_up_count: # If there exists an existing rule, then we keep this frequent set 
        if y[0] == x:
          delete = False
      
      if delete:
        del item_sets[x]
    # item_sets keep track of condset & condsUpCount
    # rules_up_count keeps track of rules count
    return item_sets, rules_up_count

  def gen_freq_rules(self, freq_set, rules_set):
    for x in rules_set.copy():
      # x is the condset (column_1, value)
      if rules_set[x]/self.transaction_length < self.min_sup: # Test for Infrequent and delete
        if x in rules_set:
          del rules_set[x]

    for x in freq_set.copy():
      delete = True
      for y in rules_set: 
        if y[0] == x: # We can find a matching ruleset 
          delete = False
      
      if delete:
        del freq_set[x]

    return freq_set, rules_set


  def gen_rules(self, freq_set, rules_set):
    # We know it's frequent, now we need see if it's confident before generating rule
    car = {}
    car_confid_supp = {}
    for x in rules_set:
      if self.total_rules_seen % 25000 == 0 and self.total_rules_seen > 0 :
        print("Total Rules Seen " + str(self.total_rules_seen))
      self.total_rules_seen += 1
      if self.total_rules_seen > 80000:
        break
      # x is the condset (column_1, value)
      if rules_set[x]/self.transaction_length >= self.min_sup: # Test for frequent again
        if x[0] not in car: # If A -> B not in car, then we assigned it into car and we check for confidence 
          confidence_temp = rules_set[x]/freq_set[x[0]]
          if confidence_temp > self.min_conf: # We want the rule to be accurate as well
            car[x[0]] = x[1] # LHS => RHS
            car_confid_supp[x] = [rules_set[x]/freq_set[x[0]] , rules_set[x]/self.transaction_length] 
        else: # TODO: Compare based on confidence to keep one rule
          cur_car_right = car[x[0]] # Get the current best rule 
          temp = (x[0], cur_car_right)
          # cur_car_confd = rules_set[temp]
          cur_car_confidence = rules_set[temp] / freq_set[x[0]]
          cond_score = rules_set[x]/freq_set[x[0]]
          if cond_score > cur_car_confidence: 
            # We change rule - And we know confidence it's greater than min_confd since original set has to be more than and this new set has more than orignial set
              car[x[0]] = x[1]
              car_confid_supp[x] = [rules_set[x]/freq_set[x[0]], rules_set[x]/self.transaction_length]     
    return car, car_confid_supp
 
  # Counts items and class occurences to determine freq 1-itemset
  def get_freq_items_list(self, full_set, first=False):
      temp_arr = []
      for x in full_set:
        temp_arr.append([x])
      return temp_arr

  def next_pass(self , freq_item, transactions):
    k = 1
    car_rules = []
    car_confid_supp_scores = []
    transaction_info = self.rule_subset_prepare(transactions)
    while freq_item: 
      candidate_sets = self.candidate_gen(freq_item, freq_item, k) 
      # Process candidate to tuple 
      candidate_sets_tuple = self.gen_cand_tuple(candidate_sets)
      rules_up_count = {}
      hash_candidates = self.generate_hash_candidates(candidate_sets)
      count = 0
      for transaction in transactions:
        # We generate all the subset found 
        results_candidate = self.rule_subset1(candidate_sets_tuple, transaction, transaction_info[count], k)
        # We do an increase in count in both support & rule
        for candidate in results_candidate: 
          temp = hash_candidates[candidate] + 1
          hash_candidates[candidate] = temp # Increase in support count
          temp1 = rules_up_count.get((candidate, results_candidate[candidate]), 0) + 1
          rules_up_count[(candidate, results_candidate[candidate])] = temp1
        count += 1
      hash_candidates = self.postprocess_hash(hash_candidates)
      freq_set, rules_up_count = self.gen_freq_rules(hash_candidates, rules_up_count)
      rules, scores = self.gen_rules(freq_set, rules_up_count)
      car_rules.append(rules)
      car_confid_supp_scores.append(scores)
      k += 1
      freq_item = [v for v in freq_set.keys()]# Assign dictionary as array
      # Candidate Pruning
      if self.total_rules_seen > 80000:
        print("Maximum Rules Reached")
        break
    return car_rules, car_confid_supp_scores

  def postprocess_hash(self, hash_cand):
    for x in hash_cand.copy():
      if hash_cand[x] == 0: # Means no support at all
        del hash_cand[x]
    return hash_cand

  def generate_hash_candidates(self, candidate_sets):
    myDict = {}
    for x in candidate_sets:
      myDict[tuple(x)] = 0
    return myDict

  def rule_subset_prepare(self, transactions):
    transaction_info = []
    for transaction in transactions: 
      temp = []
      for x in range(len(transaction)-1):
        temp.append((x, transaction[x]))
      transaction_info.append(temp)
    return transaction_info

  def rule_subset1(self, candidate_sets, transaction, transaction_row, k):
    result = {}
    for x in candidate_sets:
      satisfy = True
      for single_rule in x:
        if single_rule[1] != transaction[single_rule[0]]:
          # not a support
          satisfy = False
      if satisfy: 
        result[x] = transaction[-1]
    return result

# Helper function to convert candidate set
  def gen_cand_tuple(self, candidate_sets):
    temp = [] 
    for x in candidate_sets:
      temp.append(tuple(x))
    return temp

# Generate the candidate sets
  def candidate_gen(self, freq_item, freq_item_2, k):
    candidate_set = []    # condition of joining is that is should have k-1 elements in common 
    for x in range(len(freq_item)):
      temp = freq_item[x]
      for y in range(x+1, len(freq_item_2)):
        if x ==y: #Ignore the same key
          continue 
        temp2 = freq_item_2[y]
        # Check if the rules are the same
        if k == 1:
          temp_merged = temp[:]
          temp_merged.append(temp2[-1])
          temp_merged = tuple(temp_merged)
          if temp_merged not in candidate_set:
            candidate_set.append(temp_merged)
        elif temp[:-1] == temp2[:-1]: # Merge the 2
          temp_merged = temp + (temp2[-1],)
        # If exists in candidate set then good, else delete
          if temp_merged not in candidate_set:
            candidate_set.append(temp_merged)
    return candidate_set

  def post_process(self): # prepare data for classification 
    car_first = self.car_first
    car_all = self.all_car[:]
    scores_all = self.get_all_scores[:]
    scores_all.insert(0, self.scores)
    count = 0
    car_all.insert(0, car_first)
    myDict = {}
    for x in car_all:
      for item in x:
        myDict[(item, x[item])] = scores_all[count][(item , x[item])]
      count+=1 
    newDict = {} # Intermediate dict to format to pandas for easier sorting 
    count = 0
    for x in myDict:
      temp = myDict[x] 
      temp.insert(0,x)
      newDict[count] = temp
      count += 1

    # Sort my rules 
    new = pd.DataFrame.from_dict(newDict,  orient ='index', columns = ['rule', 'confidence', 'support'])
    df = new.sort_values(['confidence','support'], ascending=[False, False])
    return df.values 


# CBA-CB: M1


In [11]:
class CBA_CB_M1:
  def __init__(self, my_rules, transactions):
    self.my_rules = my_rules
    self.transactions = transactions

  def generate_classifier(self):
    prev_error = []
    total_error = []
    ruleList = []
    default_classes = []
    transactions_used = self.transactions[:]
    for rule in self.my_rules:
      temp = []
      actual_rule = rule[0] # Leave out the scores 
      if type(actual_rule[0][0]) is tuple: # contains only one tuple 
        single = False 
      else:
        single = True
      pred = actual_rule[-1] # gets the label of rule 
      actual_rule = actual_rule[0] # gets the LHS of rule 
      count = error = 0
      pred_right_once = False
      if len(transactions_used) == 0:
        break

      for transaction in transactions_used:
        if single:
          if transaction[actual_rule[0]] == actual_rule[1]:
            if transaction[-1] == pred: # classify at least one correctly 
              pred_right_once = True
            else:
              error += 1
            temp.append(count)
            # save the marked temp to decide whether to delete at the end or not 
        else:
          for single_col_rule in actual_rule:
            satisfy = True
            if transaction[single_col_rule[0]] != single_col_rule[1]:
              satisfy = False
          if satisfy: # Add rule 
            if transaction[-1] == pred: 
              pred_right_once = True
            else:
              error += 1
            temp.append(count) # Prepare to delete from list 
        count += 1

      if pred_right_once:
      # delete index in transactions
        new_transaction = []
        temp_counter = 0
        for item in transactions_used:
          if temp_counter not in temp:
              new_transaction.append(item)
          temp_counter += 1
        transactions_used = new_transaction[:]

        # add r to rule_list 
        ruleList.append((actual_rule, pred))

        my_dict = {}
        # add majority in remaining as default class
        for remaining in new_transaction:
          temp = my_dict.get(remaining[-1],0) + 1
          my_dict[remaining[-1]] = temp

        # find majority class 
        maj_class = [key for key,value in my_dict.items() if value == max(my_dict.values())]
        if len(maj_class) > 0:
          maj_class = maj_class[0]
        default_classes.append(maj_class)

        # calculate total number of error for default class
        default_errors = 0
        for remaining in new_transaction:
          if remaining[-1] == maj_class:
            continue
          else:
            default_errors += 1
        # print("My error " + str(error))
        # print("Default error " + str(default_errors))
        # error is the classes that this rule classify wrongly
        if len(prev_error) == 0: 
            total_error.append(default_errors + error)
            prev_error.append(error)
        else:
            total_error.append(default_errors + error + prev_error[-1])
            prev_error.append(sum(prev_error) + error)

    return ruleList, prev_error, total_error , default_classes

In [12]:
# Evaluate Test Set
def cba_m1_improved_evaluation(rules, test_set, default_class):
  my_pred = []
  true_label = []
  for test in test_set:
    true_label.append(test[-1])
    for rule in rules:
      actual_rule = rule[0] # Leave out the scores 
      if type(actual_rule[0]) is tuple: # contains only one tuple 
        single = False 
      else:
        actual_rule = tuple((actual_rule,))
        single = True

      pred = rule[-1]
      satisfy = True 
      for single_rule in actual_rule:
        if single_rule[1] != test[single_rule[0]]:
          satisfy = False
      if satisfy:
        my_pred.append(pred)# We found the rule and we want to break 
        break
      
    if not satisfy: # Means a rule was not found for this test, we resort to default 
      my_pred.append(default_class) # We predict default_class

  return my_pred, true_label


# 4) Improvement to CBA

In [13]:
from collections import Counter
# Helper Function for Ensemble Voting Amongst Weaker Class
def majority(arr):
  
    # convert array into dictionary
    freqDict = Counter(arr)
  
    # traverse dictionary and check majority element
    size = len(arr)
    for (key,val) in freqDict.items():
         if (val > (size/2)):
             return key


In [16]:
import random
# Constant Test_Set to evaluate weak models through 10CV
my_final_test = random.sample(test_data_after.tolist(), len(test_data_after)//10)

# 3 Weak Models 

In [17]:
import numpy as np
from sklearn.model_selection import KFold
import random

avg_error = 0 

for _ in range(5):
  num_weak_models = 3  # We decide on the number of "weak model"
  weak_model_pred = []
  weak_model_pred_true = []
  # We apply bagging by selecting datasets with replacement randomly for each of the weak models
  for weak_model in range(num_weak_models):
    dataset_len = len(test_data_after)
    bagging_data = [] 
    for x in range(dataset_len):    # We randomly generate the bagging_data/randomly subsample the dataset 
      idx = random.randint(0, dataset_len-1)
      bagging_data.append(test_data_after[idx])

    X = [item[:-1] for item in bagging_data]
    y = [item[-1] for item in bagging_data]

    mean_error_train = 0
    mean_rules = 0
    mean_rules_classifier = 0
    mean_classifier_error = 0
    
    kf = KFold(n_splits=10, random_state=42, shuffle=True)
    kf.get_n_splits(X)
    fold_no = 0
    weak_model_10_fold_pred = []
    weak_model_10_fold_true_pred = []

    for train_index, test_index in kf.split(X):
      # print("Currently in fold " + str(fold_no))
      my_train_data = []
      for i in train_index: 
        temp_list = np.append(X[i], y[i]) 
        my_train_data.append(temp_list.tolist())

      test_list = []
      for i in test_index:
        temp_list = np.append(X[i], y[i])
        test_list.append(temp_list.tolist())

      cba_rules = CBA_RG(0.01, my_train_data[:], 0.5) # Rule Generation happens in initialization
      # my_rules - Ruleset, Confidence, Min_Sup
      my_rules = cba_rules.post_process()
      cba_classifier = CBA_CB_M1(my_rules, my_train_data)
      ruleList, prev_error , total_error , default_classes= cba_classifier.generate_classifier()
      min_errors_idx = total_error.index(min(total_error))
      # We take rules up to that min_error rule 
      mean_classifier_error += min(total_error)
      ruleList = ruleList[:min_errors_idx+1]
      weak_pred, true_label = cba_m1_improved_evaluation(ruleList, my_final_test, default_classes[min_errors_idx])
      weak_model_10_fold_pred = weak_model_10_fold_pred + weak_pred
      weak_model_10_fold_true_pred = weak_model_10_fold_true_pred + true_label
      fold_no += 1
    print("10 CV finished")
    weak_model_pred.append(weak_model_10_fold_pred)
    weak_model_pred_true.append(weak_model_10_fold_true_pred)

  # We do majority voting ensembling for weaker model 
  error = 0 
  for x in range(len(weak_model_pred[0])): # For the test results 
    ensemble = []
    for z in range(len(weak_model_pred)): # For three model
      ensemble.append(weak_model_pred[z][x])
    # Find majority element and compare with true label 
    maj_class = majority(ensemble)
    if maj_class != weak_model_pred_true[0][x]: # Compare Majority Pred with the True Label
      error += 1 
    
  print(error/10) # This is the error for ensemble majority voting for one experiment
  avg_error += (error/10) 
print(avg_error/5) # Repeat Experiment 5 times to reduce variance due to randomness of weaker model

10 CV finished
10 CV finished
10 CV finished
25.7
10 CV finished
10 CV finished
10 CV finished
38.0
10 CV finished
10 CV finished
10 CV finished
31.8
10 CV finished
10 CV finished
10 CV finished
37.8
10 CV finished
10 CV finished
10 CV finished
36.0
33.86


# 5 Weak Models

In [18]:
import numpy as np
from sklearn.model_selection import KFold
import random
avg_error = 0 

for _ in range(5):
  num_weak_models = 5 # We decide on the number of "weak model"
  weak_model_pred = []
  weak_model_pred_true = []
  # We apply bagging by selecting datasets with replacement randomly for each of the weak models
  for weak_model in range(num_weak_models):
    # Randomly subsample the dataset 
    dataset_len = len(test_data_after)
    bagging_data = [] 
    # Keep bagging data constant 
    for x in range(dataset_len):    # We randomly generate the bagging_data
      idx = random.randint(0, dataset_len-1)
      bagging_data.append(test_data_after[idx])

    X = [item[:-1] for item in bagging_data]
    y = [item[-1] for item in bagging_data]

    mean_error_train = 0
    mean_rules = 0
    mean_rules_classifier = 0
    mean_classifier_error = 0
    
    kf = KFold(n_splits=10, random_state=42, shuffle=True)
    kf.get_n_splits(X)
    fold_no = 0
    weak_model_10_fold_pred = []
    weak_model_10_fold_true_pred = []

    for train_index, test_index in kf.split(X):
      # print("Currently in fold " + str(fold_no))
      my_train_data = []
      for i in train_index: 
        temp_list = np.append(X[i], y[i]) 
        my_train_data.append(temp_list.tolist())

      test_list = []
      for i in test_index:
        temp_list = np.append(X[i], y[i])
        test_list.append(temp_list.tolist())

      cba_rules = CBA_RG(0.01, my_train_data[:], 0.5) # Rule Generation happens in initialization
      
      # my_rules - Ruleset, Confidence, Min_Sup
      my_rules = cba_rules.post_process() 
      cba_classifier = CBA_CB_M1(my_rules, my_train_data)
      ruleList, prev_error , total_error , default_classes= cba_classifier.generate_classifier()
      min_errors_idx = total_error.index(min(total_error))
      # We take rules up to that min_error rule 
      mean_classifier_error += min(total_error)
      ruleList = ruleList[:min_errors_idx+1]
      weak_pred, true_label = cba_m1_improved_evaluation(ruleList, my_final_test, default_classes[min_errors_idx])
      weak_model_10_fold_pred = weak_model_10_fold_pred + weak_pred
      weak_model_10_fold_true_pred = weak_model_10_fold_true_pred + true_label
      fold_no += 1
    print("10 CV finished")
    # 10 fold CV done - We add to weak_model_pred
    weak_model_pred.append(weak_model_10_fold_pred)
    weak_model_pred_true.append(weak_model_10_fold_true_pred)

  # We do majority voting ensembling for weaker model 
  error = 0 
  for x in range(len(weak_model_pred[0])): # For the test results 
    ensemble = []
    for z in range(len(weak_model_pred)): # For five model
      ensemble.append(weak_model_pred[z][x])
    # Find majority element and compare with true label
    maj_class = majority(ensemble)
    if maj_class != weak_model_pred_true[0][x]: # Compare Majority voting for one experiment
      error += 1 
    
  print(error/10)
  avg_error += (error/10) 
print(avg_error/5) # Repeat Experiment 5 times to reduce variance due to randomness of weaker model

10 CV finished
10 CV finished
10 CV finished
10 CV finished
10 CV finished
34.2
10 CV finished
10 CV finished
10 CV finished
10 CV finished
10 CV finished
39.7
10 CV finished
10 CV finished
10 CV finished
10 CV finished
10 CV finished
34.6
10 CV finished
10 CV finished
10 CV finished
10 CV finished
10 CV finished
32.6
10 CV finished
10 CV finished
10 CV finished
10 CV finished
10 CV finished
37.8
35.779999999999994


# 7 Weak Models

In [19]:
# Split Data into 10 sets

import numpy as np
from sklearn.model_selection import KFold
import random
avg_error = 0 

for _ in range(5):
  num_weak_models = 7 # We decide on the number of "weak model"
  weak_model_pred = []
  weak_model_pred_true = []
  # We apply bagging by selecting datasets with replacement randomly for each of the weak models
  for weak_model in range(num_weak_models): # We randomly generate the bagging_data/randomly subsample the dataset 
    dataset_len = len(test_data_after)
    bagging_data = [] 
    for x in range(dataset_len):    # We randomly generate the bagging_data
      idx = random.randint(0, dataset_len-1)
      bagging_data.append(test_data_after[idx])

    X = [item[:-1] for item in bagging_data]
    y = [item[-1] for item in bagging_data]

    mean_error_train = 0
    mean_rules = 0
    mean_rules_classifier = 0
    mean_classifier_error = 0
    
    kf = KFold(n_splits=10, random_state=42, shuffle=True)
    kf.get_n_splits(X)
    fold_no = 0
    weak_model_10_fold_pred = []
    weak_model_10_fold_true_pred = []

    for train_index, test_index in kf.split(X):
      # print("Currently in fold " + str(fold_no))
      my_train_data = []
      for i in train_index: 
        temp_list = np.append(X[i], y[i]) 
        my_train_data.append(temp_list.tolist())

      test_list = []
      for i in test_index:
        temp_list = np.append(X[i], y[i])
        test_list.append(temp_list.tolist())

      cba_rules = CBA_RG(0.01, my_train_data[:], 0.5) # Rule Generation happens in initialization
      # my_rules - Ruleset, Confidence, Min_Sup
      my_rules = cba_rules.post_process() 
      cba_classifier = CBA_CB_M1(my_rules, my_train_data)
      ruleList, prev_error , total_error , default_classes= cba_classifier.generate_classifier()
      min_errors_idx = total_error.index(min(total_error))
      # We take rules up to that min_error rule 
      mean_classifier_error += min(total_error)
      ruleList = ruleList[:min_errors_idx+1]
      weak_pred, true_label = cba_m1_improved_evaluation(ruleList, my_final_test, default_classes[min_errors_idx])
      weak_model_10_fold_pred = weak_model_10_fold_pred + weak_pred
      weak_model_10_fold_true_pred = weak_model_10_fold_true_pred + true_label
      fold_no += 1
    print("10 CV finished")
    # 10 fold CV done - We add to weak_model_pred
    weak_model_pred.append(weak_model_10_fold_pred)
    weak_model_pred_true.append(weak_model_10_fold_true_pred)
  # We do majority voting ensembling for weaker model and calculate error 
  error = 0 
  for x in range(len(weak_model_pred[0])): 
    ensemble = []
    for z in range(len(weak_model_pred)): # For 7 model
      ensemble.append(weak_model_pred[z][x])
    # Find majority element and compare with weak_model_pred_true[0][x]
    maj_class = majority(ensemble)
    if maj_class != weak_model_pred_true[0][x]:
      error += 1 
    
  print(error/10)
  avg_error += (error/10) 
print(avg_error/5)

10 CV finished
10 CV finished
10 CV finished
10 CV finished
10 CV finished
10 CV finished
10 CV finished
36.6
10 CV finished
10 CV finished
10 CV finished
10 CV finished
10 CV finished
10 CV finished
10 CV finished
34.9
10 CV finished
10 CV finished
10 CV finished
10 CV finished
10 CV finished
10 CV finished
10 CV finished
35.5
10 CV finished
10 CV finished
10 CV finished
10 CV finished
10 CV finished
10 CV finished
10 CV finished
30.8
10 CV finished
10 CV finished
10 CV finished
10 CV finished
10 CV finished
10 CV finished
10 CV finished
29.8
33.52


# Original 1 Weak Model

In [20]:
# Split Data into 10 sets

import numpy as np
from sklearn.model_selection import KFold
import random
avg_error = 0 

# We apply bagging by selecting datasets with replacement randomly ~ 
# We then train this dataset with 10CV 
for _ in range(5):
  # We decide on the number of "weak model"
  num_weak_models = 1
  weak_model_pred = []
  weak_model_pred_true = []
  # We shall sample 3 diff datasets each time 
  for weak_model in range(num_weak_models):
    bagging_data = test_data_after[:]
    X = [item[:-1] for item in bagging_data]
    y = [item[-1] for item in bagging_data]

    mean_error_train = 0
    mean_rules = 0
    mean_rules_classifier = 0
    mean_classifier_error = 0
    
    kf = KFold(n_splits=10, random_state=42, shuffle=True)
    kf.get_n_splits(X)
    fold_no = 0
    weak_model_10_fold_pred = []
    weak_model_10_fold_true_pred = []

    for train_index, test_index in kf.split(X):
      # print("Currently in fold " + str(fold_no))
      my_train_data = []
      for i in train_index: 
        temp_list = np.append(X[i], y[i]) 
        my_train_data.append(temp_list.tolist())

      test_list = []
      for i in test_index:
        temp_list = np.append(X[i], y[i])
        test_list.append(temp_list.tolist())

      cba_rules = CBA_RG(0.01, my_train_data[:], 0.5) # Rule Generation happens in initialization
      
      # my_rules - Ruleset, Confidence, Min_Sup
      my_rules = cba_rules.post_process() 
      cba_classifier = CBA_CB_M1(my_rules, my_train_data)
      ruleList, prev_error , total_error , default_classes= cba_classifier.generate_classifier()
      min_errors_idx = total_error.index(min(total_error))
      # We take rules up to that min_error rule 
      mean_classifier_error += min(total_error)
      ruleList = ruleList[:min_errors_idx+1]
      weak_pred, true_label = cba_m1_improved_evaluation(ruleList, my_final_test, default_classes[min_errors_idx])
      # print("Printing my prediciotn")
      weak_model_10_fold_pred = weak_model_10_fold_pred + weak_pred
      weak_model_10_fold_true_pred = weak_model_10_fold_true_pred + true_label
      fold_no += 1
    print("10 CV finished")
    weak_model_pred.append(weak_model_10_fold_pred)
    weak_model_pred_true.append(weak_model_10_fold_true_pred)

  # After all the iteration, we then do ensemble majority voting
  error = 0 
  for x in range(len(weak_model_pred[0])): # For the test results 
    ensemble = []
    for z in range(len(weak_model_pred)): # For 1 model
      ensemble.append(weak_model_pred[z][x])
    # Find majority element and compare with weak_model_pred_true[0][x]
    maj_class = majority(ensemble)
    if maj_class != weak_model_pred_true[0][x]:
      error += 1 
    
  print(error/10)
  avg_error += (error/10) 
  # If 1 Model -> 38 errors
print(avg_error/5)

10 CV finished
41.4
10 CV finished
41.4
10 CV finished
41.4
10 CV finished
41.4
10 CV finished
41.4
41.4
